In [1]:
import pandas as pd
ogsql_df = pd.read_csv('311_Service_Requests_from_2010_to_Present_20251115.csv')

# Display the first 10 rows
print(ogsql_df.head(10))

   Unique Key            Created Date Closed Date Agency  \
0    66332327  09/30/2025 11:39:28 PM         NaN    TLC   
1    66330808  09/30/2025 11:06:39 PM         NaN    TLC   
2    66322041  09/30/2025 10:03:37 PM         NaN    TLC   
3    66324855  09/30/2025 09:23:14 PM         NaN    TLC   
4    66332465  09/30/2025 08:54:46 PM         NaN    TLC   
5    66332328  09/30/2025 08:18:38 PM         NaN    TLC   
6    66330810  09/30/2025 07:23:55 PM         NaN    TLC   
7    66321865  09/30/2025 06:33:20 PM         NaN    TLC   
8    66329299  09/30/2025 06:13:13 PM         NaN    TLC   
9    66330806  09/30/2025 05:52:01 PM         NaN    TLC   

                     Agency Name  Complaint Type  \
0  Taxi and Limousine Commission  Taxi Complaint   
1  Taxi and Limousine Commission  Taxi Complaint   
2  Taxi and Limousine Commission  Taxi Complaint   
3  Taxi and Limousine Commission  Taxi Complaint   
4  Taxi and Limousine Commission  Taxi Complaint   
5  Taxi and Limousine Commi

In [2]:
# Check null
missing_values = ogsql_df.isnull().sum()

missing_percentage = (missing_values / len(ogsql_df)) * 100

print(f"Missing values (%):\n{missing_percentage}")

Missing values (%):
Unique Key                          0.000000
Created Date                        0.000000
Closed Date                        16.239352
Agency                              0.000000
Agency Name                         0.000000
Complaint Type                      0.000000
Descriptor                          0.000000
Location Type                      15.425690
Incident Zip                        3.966606
Incident Address                    3.941179
Street Name                         3.941179
Cross Street 1                      3.979319
Cross Street 2                      4.047125
Intersection Street 1               3.979319
Intersection Street 2               4.047125
Address Type                        3.949655
City                               19.794889
Landmark                           19.811840
Facility Type                     100.000000
Status                              0.000000
Due Date                          100.000000
Resolution Description             

In [3]:
ogsql_df.columns = (
    ogsql_df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("/", "_")
        .str.replace("-", "_")
)
df = ogsql_df.copy()   # 保留原始版本

In [4]:
cols_to_drop = [
    "facility_type",
    "due_date",
    "vehicle_type",
    "taxi_company_borough",
    "bridge_highway_name",
    "bridge_highway_direction",
    "road_ramp",
    "bridge_highway_segment"
]

df = df.drop(columns=cols_to_drop, errors="ignore")

In [5]:
df = df.dropna(subset=["latitude", "longitude"])
date_cols = [
    "created_date",
    "closed_date",
    "resolution_action_updated_date"
]

for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

/var/folders/3s/_7ghh_l946lcs5d2bykq1gnc0000gn/T/ipykernel_93869/3398373530.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")
/var/folders/3s/_7ghh_l946lcs5d2bykq1gnc0000gn/T/ipykernel_93869/3398373530.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")
/var/folders/3s/_7ghh_l946lcs5d2bykq1gnc0000gn/T/ipykernel_93869/3398373530.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")


In [7]:
text_cols = [
    "agency",
    "agency_name",
    "complaint_type",
    "descriptor",
    "city",
    "borough",
    "location_type"
]

for col in text_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.title()

In [6]:
df["location_geojson"] = df.apply(
    lambda row: {
        "type": "Point",
        "coordinates": [row["longitude"], row["latitude"]]
    },
    axis=1
)

In [7]:
df = df.drop(columns=["location"], errors="ignore")
df = df.replace({pd.NaT: None})
records = df.to_dict("records")
print("清洗完成！共有记录数：", len(df))
print("可以直接写入 MongoDB 的格式：records")

清洗完成！共有记录数： 22663
可以直接写入 MongoDB 的格式：records


In [19]:
records[:3]

[{'unique_key': 66332327,
  'created_date': Timestamp('2025-09-30 23:39:28'),
  'closed_date': None,
  'agency': 'Tlc',
  'agency_name': 'Taxi And Limousine Commission',
  'complaint_type': 'Taxi Complaint',
  'descriptor': 'Driver Complaint - Passenger',
  'location_type': 'Street',
  'incident_zip': 11430.0,
  'incident_address': 'JOHN F KENNEDY AIRPORT',
  'street_name': 'JOHN F KENNEDY AIRPORT',
  'cross_street_1': 'BEND',
  'cross_street_2': 'AIRTRAIN-HOWARD BCH/JAMAICA LINE',
  'intersection_street_1': 'BEND',
  'intersection_street_2': 'AIRTRAIN-HOWARD BCH/JAMAICA LINE',
  'address_type': 'PLACE',
  'city': 'Jamaica',
  'landmark': 'JOHN F KENNEDY AIRPORT',
  'status': 'In Progress',
  'resolution_description': None,
  'resolution_action_updated_date': None,
  'community_board': '83 QUEENS',
  'bbl': 4142600001.0,
  'borough': 'Queens',
  'x_coordinate_(state_plane)': '1,043,001',
  'y_coordinate_(state_plane)': '175,548',
  'open_data_channel_type': 'ONLINE',
  'park_facility_n

In [15]:
pip install pymongo


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
from pymongo import MongoClient

# 连接 Docker 里的 MongoDB
client = MongoClient("mongodb://localhost:27017/")
# 创建数据库（不存在会自动创建）
db = client["nyc311"]
# 创建集合
collection = db["requests"]
print("Connected to MongoDB!")

Connected to MongoDB!


In [21]:
#插入
collection.insert_many(records)

InsertManyResult([ObjectId('692c93056b49622f47e3e10c'), ObjectId('692c93056b49622f47e3e10d'), ObjectId('692c93056b49622f47e3e10e'), ObjectId('692c93056b49622f47e3e10f'), ObjectId('692c93056b49622f47e3e110'), ObjectId('692c93056b49622f47e3e111'), ObjectId('692c93056b49622f47e3e112'), ObjectId('692c93056b49622f47e3e113'), ObjectId('692c93056b49622f47e3e114'), ObjectId('692c93056b49622f47e3e115'), ObjectId('692c93056b49622f47e3e116'), ObjectId('692c93056b49622f47e3e117'), ObjectId('692c93056b49622f47e3e118'), ObjectId('692c93056b49622f47e3e119'), ObjectId('692c93056b49622f47e3e11a'), ObjectId('692c93056b49622f47e3e11b'), ObjectId('692c93056b49622f47e3e11c'), ObjectId('692c93056b49622f47e3e11d'), ObjectId('692c93056b49622f47e3e11e'), ObjectId('692c93056b49622f47e3e11f'), ObjectId('692c93056b49622f47e3e120'), ObjectId('692c93056b49622f47e3e121'), ObjectId('692c93056b49622f47e3e122'), ObjectId('692c93056b49622f47e3e123'), ObjectId('692c93056b49622f47e3e124'), ObjectId('692c93056b49622f47e3e1

In [24]:
collection.count_documents({})

22663

In [25]:
collection.find_one()

{'_id': ObjectId('692c93056b49622f47e3e10c'),
 'unique_key': 66332327,
 'created_date': datetime.datetime(2025, 9, 30, 23, 39, 28),
 'closed_date': None,
 'agency': 'Tlc',
 'agency_name': 'Taxi And Limousine Commission',
 'complaint_type': 'Taxi Complaint',
 'descriptor': 'Driver Complaint - Passenger',
 'location_type': 'Street',
 'incident_zip': 11430.0,
 'incident_address': 'JOHN F KENNEDY AIRPORT',
 'street_name': 'JOHN F KENNEDY AIRPORT',
 'cross_street_1': 'BEND',
 'cross_street_2': 'AIRTRAIN-HOWARD BCH/JAMAICA LINE',
 'intersection_street_1': 'BEND',
 'intersection_street_2': 'AIRTRAIN-HOWARD BCH/JAMAICA LINE',
 'address_type': 'PLACE',
 'city': 'Jamaica',
 'landmark': 'JOHN F KENNEDY AIRPORT',
 'status': 'In Progress',
 'resolution_description': None,
 'resolution_action_updated_date': None,
 'community_board': '83 QUEENS',
 'bbl': 4142600001.0,
 'borough': 'Queens',
 'x_coordinate_(state_plane)': '1,043,001',
 'y_coordinate_(state_plane)': '175,548',
 'open_data_channel_type':

In [9]:
#Step 1 — 从 MongoDB 取出数据（只取纬经度）
from pymongo import MongoClient
import pandas as pd

client = MongoClient("mongodb://localhost:27017/")
db = client["nyc311"]
collection = db["requests"]

# 只取 latitude / longitude 提高速度
cursor = collection.find({}, {"latitude": 1, "longitude": 1, "_id": 0}).limit(200000)
df = pd.DataFrame(list(cursor))

df.head()

,latitude,longitude
0,40.648320,-73.788281
1,40.648320,-73.788281
2,40.829645,-73.926505
3,40.648320,-73.788281
4,40.752389,-73.944944


In [10]:
cursor = collection.find(
    {},
    {
        "latitude": 1,
        "longitude": 1,
        "descriptor": 1,    # ★ 必须选这个字段
        "_id": 0
    }
).limit(200000)

df = pd.DataFrame(list(cursor))


In [50]:
df["descriptor"].unique()

array(['Driver Complaint - Passenger', 'Vehicle Complaint',
       'Driver Complaint - Non Passenger', 'Driver Report - Passenger',
       'Car Service Company Report', 'Driver Complaint',
       'Car Service Company Complaint', 'Driver Report - Non Passenger',
       'Jewelry'], dtype=object)

In [54]:
df = df.rename(columns={
    "lat": "latitude",
    "lng": "longitude"
})

In [55]:
df["descriptor"] = df["descriptor"].astype(str).str.strip()

In [11]:
#清洗异常点
df = df.dropna(subset=["latitude", "longitude"])
df = df[(df.latitude > 35) & (df.latitude < 45)]
df = df[(df.longitude > -80) & (df.longitude < -70)]

In [12]:
import folium
from folium.plugins import HeatMap

# 分类函数
def classify_descriptor(desc):
    desc = str(desc).lower()

    if "driver complaint" in desc or "driver report" in desc:
        return "Driver Behavior Issues"
    elif "vehicle complaint" in desc:
        return "Vehicle Issues"
    elif "car service company" in desc:
        return "Company Service Issues"
    else:
        return "Other"

df["category"] = df["descriptor"].apply(classify_descriptor)

# 初始化地图
m = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

# 1. Overall 图层
HeatMap(
    df[['latitude','longitude']].dropna().values.tolist(),
    radius=8,
    blur=6,
    name="Overall Hotspot"
).add_to(m)

# 2. 大类图层
for cat in df["category"].unique():
    sub_df = df[df["category"] == cat].dropna(subset=["latitude", "longitude"])

    if len(sub_df) == 0:
        continue

    HeatMap(
        sub_df[['latitude','longitude']].values.tolist(),
        radius=8,
        blur=6,
        name=f"Category: {cat}"
    ).add_to(m)

# 3. 小类 Descriptor 图层
for desc in df["descriptor"].unique():
    sub_df = df[df["descriptor"] == desc].dropna(subset=["latitude", "longitude"])

    if len(sub_df) < 10:
        continue

    HeatMap(
        sub_df[['latitude','longitude']].values.tolist(),
        radius=8,
        blur=6,
        name=f"Descriptor: {desc}"
    ).add_to(m)

# 图层控制器
folium.LayerControl(collapsed=False).add_to(m)

m.save("NYC_311_Taxi_Category_Descriptor_Map.html")
print("地图已生成：NYC_311_Taxi_Category_Descriptor_Map.html")


地图已生成：NYC_311_Taxi_Category_Descriptor_Map.html
